In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
from sklearn.utils import shuffle
from scipy.stats import ttest_ind

# Загрузка данных
url="Netology_A_B-testing_HW_1.csv"
data=pd.read_csv(url, names=['id','date','user_id','hour','app_opened','price_seen','order_made','surge','ride_completed',\
                             'user_cancelled','age','os','city_center_order','order_class','distance','rfm'],
                skiprows=1)
data.head()

,id,date,user_id,hour,app_opened,price_seen,order_made,surge,ride_completed,user_cancelled,age,os,city_center_order,order_class,distance,rfm
0,0,2020-05-21,867689,12,1,1,1,no surge,1,0,20,iOS,0,business,7.982135,low
1,1,2020-05-23,752172,5,1,1,1,no surge,1,0,37,Android,1,economy,2.908468,high
2,2,2020-05-20,486559,15,1,1,1,no surge,1,0,47,Android,0,comfort,7.224614,low
3,3,2020-05-19,304024,0,1,1,1,no surge,1,0,59,Android,1,economy,1.874349,low
4,4,2020-05-23,139420,0,1,1,1,no surge,1,0,19,Android,0,business,10.704778,medium


Выберем следующие особенности и придумаем для них А/Б тест:
- Android => более высокие юзерские отмены
- Comfort & Business классы => более низкие юзерские отмены, более длинные поездки

In [ ]:
res1 = {}
res2= {}
res3 = {}

In [142]:
res1['Android'] = data[(data['os']=='Android') & (data['user_cancelled']==1)]['os'].count()
res1['iOS'] = data[(data['os']=='iOS') & (data['user_cancelled']==1)]['os'].count()
print ('Android =',res1['Android'], ', iOS =', res1['iOS'])

Android = 7353 , iOS = 3916


In [143]:
res2['business'] = data[(data['order_class']=='business') & (data['user_cancelled']==1)]['order_class'].count()
res2['comfort'] = data[(data['order_class']=='comfort') & (data['user_cancelled']==1)]['order_class'].count()
res2['economy'] = data[(data['order_class']=='economy') & (data['user_cancelled']==1)]['order_class'].count()
print ('business =',res2['business'], ', comfort =', res2['comfort'], ', economy =', res2['economy'])

business = 1011 , comfort = 4049 , economy = 6209


In [146]:
res3['business'] = data[(data['order_class']=='business')]['distance'].sum()
res3['comfort'] = data[(data['order_class']=='comfort')]['distance'].sum()
res3['economy'] = data[(data['order_class']=='economy')]['distance'].sum()
print ('business =',(res3['business']/res2['business']).round(),
       ', comfort =', (res3['comfort']/res2['comfort']).round(),
       ', economy =', (res3['economy']/res2['economy']).round())

business = 128.0 , comfort = 47.0 , economy = 27.0


Как можно увидеть выше на основе выборок, данные особенности верны:
- Пользователи с ОС Android чаще, чем пользователи iOS отменяют поездки;
- Пользователи Бизнесс класса и Комфорта реже отменяют поездки и делают более длинные поездки

Выдвинем следующие гипотезы:
- Пользователи с Android чаще пользуются Эконом классом, чем другими классами
- У пользователей iOS более длинные поездки

Теперь отфильтруем данные в соответствии с гипотезами:

In [149]:
data1 = data[(data['os']=='Android')]
data2 = data[(data['os']=='iOS')]

Далее поделим каждую выборку случайным образом на 2

In [156]:
cnt1 = data1['os'].count()
half1 = round(cnt1/2).astype('int64')
data1 = shuffle(data1)
data1a = data1[0:half1]
data1b = data1[half1:cnt1]

In [157]:
cnt2 = data2['os'].count()
half2 = round(cnt2/2).astype('int64')
data2 = shuffle(data2)
data2a = data2[0:half2]
data2b = data2[half2:cnt2]

Теперь проведем t-тесты

In [163]:
ttest_ind(data1a['user_id'], data1b['user_id'])

Ttest_indResult(statistic=-0.5559606700392781, pvalue=0.5782399356497588)